In [1]:
pip install bech32 base58

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import svm

In [3]:
import requests
import pandas as pd

# Function to get Bitcoin addresses
def get_btc_addresses(limit=1250):
    btc_addresses = set()
    url = 'https://blockchain.info/latestblock'
    response = requests.get(url)
    latest_block = response.json()
    block_hash = latest_block['hash']

    while len(btc_addresses) < limit:
        block_url = f'https://blockchain.info/block/{block_hash}?format=json'
        response = requests.get(block_url)
        block_data = response.json()

        for tx in block_data['tx']:
            for out in tx['out']:
                if 'addr' in out:
                    btc_addresses.add(out['addr'])
                    if len(btc_addresses) >= limit:
                        break
            if len(btc_addresses) >= limit:
                break
        block_hash = block_data['prev_block']

    return list(btc_addresses)[:limit]

# Function to get Ethereum addresses
def get_eth_addresses(limit=1250):
    eth_addresses = set()
    url = 'https://api.etherscan.io/api?module=account&action=txlist&address=0xde0B295669a9FD93d5F28D9Ec85E40f4cb697BAe&startblock=0&endblock=99999999&sort=asc&apikey=E1TRQRSFP237WHQG11I344FTW6WH1XG4WE'
    response = requests.get(url)
    transactions = response.json()['result']

    for tx in transactions:
        eth_addresses.add(tx['from'])
        eth_addresses.add(tx['to'])
        if len(eth_addresses) >= limit:
            break

    return list(eth_addresses)[:limit]

# Function to get TRON addresses
def get_trx_addresses(limit=1250):
    trx_addresses = set()
    url = 'https://api.tronscan.org/api/transaction?sort=-timestamp&count=true&limit=100&start=0&apikey=7c31c3b5-939f-4d58-a66b-5e54bb970efd'
    response = requests.get(url)
    transactions = response.json()['data']

    for tx in transactions:
        trx_addresses.add(tx['ownerAddress'])
        trx_addresses.add(tx['toAddress'])
        if len(trx_addresses) >= limit:
            break

    return list(trx_addresses)[:limit]



# Main function to gather addresses and save to DataFrame
def create_crypto_address_dataset():
    btc_addresses = get_btc_addresses()
    eth_addresses = get_eth_addresses()
    trx_addresses = get_trx_addresses()
    #bsc_addresses = get_bsc_addresses()

    data = {
        'address': btc_addresses + eth_addresses + trx_addresses,
        'type': ['BTC'] * len(btc_addresses) + ['ETH'] * len(eth_addresses) + ['TRX'] * len(trx_addresses)
    }

    df = pd.DataFrame(data)
    return df

if __name__ == "__main__":
    df = create_crypto_address_dataset()


    print(df)


                                         address type
0     bc1q5mehm8skgz30j7zur4nruyxffkf8jntmhkuuf9  BTC
1     bc1qdlutwqkwpaz4kwhf4napdy5d6h4k8g7nnrsea6  BTC
2             1PEZAEEmptFtcT3B1TVNxNHfwJFjr6N41n  BTC
3     bc1qfncmp76klszdgy5juuujd0kn6zhvza0prmasxr  BTC
4     bc1qyd09rkye7vrqgdr498p0pdf8hd60jnghkgmz6j  BTC
...                                          ...  ...
2581          TXPLfbfPR8EkZryyBfN61XVPdtnqdLt1CW  TRX
2582          TTFJJ749zgYRowsdque6DXtRY1KVavZKri  TRX
2583          TM7RnN8VoRwatq4GVdgTcuoAqPqi6LRLxW  TRX
2584          TDHT8sfWXeeQPsAH4HFKguq2WJgwtuHg8S  TRX
2585          TPv3Hqhk37FkFkr2j535HwbSR5fPbbD62F  TRX

[2586 rows x 2 columns]


In [4]:
import re
import base58
import hashlib
import bech32
import pandas as pd

def is_valid_legacy_or_p2sh_address(address):
    try:
        decoded = base58.b58decode(address)
        payload, checksum = decoded[:-4], decoded[-4:]
        hash1 = hashlib.sha256(payload).digest()
        hash2 = hashlib.sha256(hash1).digest()
        return checksum == hash2[:4]
    except Exception as e:
        return False

def is_valid_bech32_address(address):
    try:
        hrp, data = bech32.bech32_decode(address)
        if hrp != 'bc' and hrp != 'tb':
            return False
        if data is None:
            return False
        return True
    except Exception as e:
        return False

def is_valid_eth_address(address):
    if not re.match(r'^0x[a-fA-F0-9]{40}$', address):
        return False
    if address != address.lower() and address != address.upper():
        return check_eth_checksum(address)
    return True

def check_eth_checksum(address):
    address = address.replace('0x', '')
    address_hash = hashlib.sha3_256(address.lower().encode('utf-8')).hexdigest()
    for i in range(40):
        if (address_hash[i] >= '8' and address[i].upper() != address[i]) or (address_hash[i] < '8' and address[i].lower() != address[i]):
            return False
    return True

def validate_tron_address(address):
    if len(address) == 34:
        return True
    return False

def extract_features(address):
    features = {
        'length': len(address),
        'character_set': set(address),
        'prefix': address[:2],
        'is_valid': False,
        'type': 'unknown'
    }

    if address.startswith(('1', '3', 'bc1')):
        features['type'] = 'Bitcoin'
        if address.startswith('1') or address.startswith('3'):
            features['is_valid'] = is_valid_legacy_or_p2sh_address(address)
        elif address.startswith('bc1'):
            features['is_valid'] = is_valid_bech32_address(address)
    elif address.startswith('0x'):
        features['type'] = 'Ethereum'
        features['is_valid'] = is_valid_eth_address(address)
    elif validate_tron_address(address):
        features['type'] = 'TRON'
        features['is_valid'] = True

    return features


# Add checksum validation feature
df['checksum_validation'] = df['address'].apply(lambda x: extract_features(x)['is_valid'])

# Display the updated DataFrame
print(df)


                                         address type  checksum_validation
0     bc1q5mehm8skgz30j7zur4nruyxffkf8jntmhkuuf9  BTC                 True
1     bc1qdlutwqkwpaz4kwhf4napdy5d6h4k8g7nnrsea6  BTC                 True
2             1PEZAEEmptFtcT3B1TVNxNHfwJFjr6N41n  BTC                 True
3     bc1qfncmp76klszdgy5juuujd0kn6zhvza0prmasxr  BTC                 True
4     bc1qyd09rkye7vrqgdr498p0pdf8hd60jnghkgmz6j  BTC                 True
...                                          ...  ...                  ...
2581          TXPLfbfPR8EkZryyBfN61XVPdtnqdLt1CW  TRX                 True
2582          TTFJJ749zgYRowsdque6DXtRY1KVavZKri  TRX                 True
2583          TM7RnN8VoRwatq4GVdgTcuoAqPqi6LRLxW  TRX                 True
2584          TDHT8sfWXeeQPsAH4HFKguq2WJgwtuHg8S  TRX                 True
2585          TPv3Hqhk37FkFkr2j535HwbSR5fPbbD62F  TRX                 True

[2586 rows x 3 columns]


In [5]:
df.head()

,address,type,checksum_validation
0,bc1q5mehm8skgz30j7zur4nruyxffkf8jntmhkuuf9,BTC,True
1,bc1qdlutwqkwpaz4kwhf4napdy5d6h4k8g7nnrsea6,BTC,True
2,1PEZAEEmptFtcT3B1TVNxNHfwJFjr6N41n,BTC,True
3,bc1qfncmp76klszdgy5juuujd0kn6zhvza0prmasxr,BTC,True
4,bc1qyd09rkye7vrqgdr498p0pdf8hd60jnghkgmz6j,BTC,True


In [6]:
df.tail()

,address,type,checksum_validation
2581,TXPLfbfPR8EkZryyBfN61XVPdtnqdLt1CW,TRX,True
2582,TTFJJ749zgYRowsdque6DXtRY1KVavZKri,TRX,True
2583,TM7RnN8VoRwatq4GVdgTcuoAqPqi6LRLxW,TRX,True
2584,TDHT8sfWXeeQPsAH4HFKguq2WJgwtuHg8S,TRX,True
2585,TPv3Hqhk37FkFkr2j535HwbSR5fPbbD62F,TRX,True


In [25]:
# Convert addresses into numerical features
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 5))  # Example with character n-grams
X = vectorizer.fit_transform(df['address'],df['checksum_validation'])

# Encode target labels
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(df['type'])

In [14]:
print(X)

  (0, 95021)	1
  (0, 97869)	1
  (0, 15183)	1
  (0, 165557)	1
  (0, 48287)	1
  (0, 153602)	1
  (0, 122253)	1
  (0, 139104)	1
  (0, 152977)	1
  (0, 73182)	1
  (0, 176400)	1
  (0, 146779)	1
  (0, 136824)	1
  (0, 204059)	1
  (0, 24848)	1
  (0, 6340)	1
  (0, 141992)	1
  (0, 65823)	1
  (0, 206872)	1
  (0, 184920)	1
  (0, 170743)	1
  (0, 40292)	1
  (0, 158874)	1
  (0, 173568)	1
  (0, 185821)	1
  :	:
  (2585, 167549)	1
  (2585, 138870)	1
  (2585, 145357)	1
  (2585, 27885)	1
  (2585, 64033)	1
  (2585, 131337)	1
  (2585, 146717)	1
  (2585, 131374)	1
  (2585, 147894)	1
  (2585, 170577)	1
  (2585, 23200)	1
  (2585, 141825)	1
  (2585, 42872)	1
  (2585, 27083)	1
  (2585, 48002)	1
  (2585, 140264)	1
  (2585, 191083)	1
  (2585, 97104)	1
  (2585, 177117)	1
  (2585, 170907)	1
  (2585, 47776)	1
  (2585, 131809)	1
  (2585, 161901)	1
  (2585, 94928)	1
  (2585, 95564)	1


In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:

model = RandomForestClassifier(n_estimators=100, criterion='entropy',random_state=42)
model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

In [17]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       253
           1       1.00      1.00      1.00       251
           2       0.00      0.00      0.00        14

    accuracy                           0.97       518
   macro avg       0.65      0.67      0.66       518
weighted avg       0.95      0.97      0.96       518



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
accuracy = accuracy_score(Y_test, Y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.972972972972973


In [28]:
import re
import base58
import hashlib
import bech32

def is_valid_legacy_or_p2sh_address(address):
    try:
        decoded = base58.b58decode(address)
        payload, checksum = decoded[:-4], decoded[-4:]
        hash1 = hashlib.sha256(payload).digest()
        hash2 = hashlib.sha256(hash1).digest()
        return checksum == hash2[:4]
    except Exception as e:
        return False

def is_valid_bech32_address(address):
    try:
        hrp, data = bech32.bech32_decode(address)
        if hrp != 'bc' and hrp != 'tb':
            return False
        if data is None:
            return False
        return True
    except Exception as e:
        return False

def is_valid_eth_address(address):
    if not re.match(r'^0x[a-fA-F0-9]{40}$', address):
        return False
    if address != address.lower() and address != address.upper():
        return check_eth_checksum(address)
    return True

def check_eth_checksum(address):
    address = address.replace('0x', '')
    address_hash = hashlib.sha3_256(address.lower().encode('utf-8')).hexdigest()
    for i in range(40):
        if (address_hash[i] >= '8' and address[i].upper() != address[i]) or (address_hash[i] < '8' and address[i].lower() != address[i]):
            return False
    return True

def validate_tron_address(address):
    if len(address) == 34:
        return True
    return False

def extract_features(address):
    features = {
        'length': len(address),
        'character_set': set(address),
        'prefix': address[:2],
        'is_valid': False,
        'type': 'unknown'
    }

    if address.startswith(('1', '3', 'bc1')):
        features['type'] = 'Bitcoin'
        if address.startswith('1') or address.startswith('3'):
            features['is_valid'] = is_valid_legacy_or_p2sh_address(address)
        elif address.startswith('bc1'):
            features['is_valid'] = is_valid_bech32_address(address)
    elif address.startswith('0x'):
        features['type'] = 'Ethereum'
        features['is_valid'] = is_valid_eth_address(address)
    elif validate_tron_address(address):
        features['type'] = 'TRON'
        features['is_valid'] = True

    return features

def validate_and_extract_address_features(address):
    features = extract_features(address)
    return features

# Get user input
#user_address = input("Please enter a cryptocurrency address: ")

# Validate and extract features


# Print the validation result
#print("Address is", "valid" if address_features['is_valid'] else "invalid")


add=str(input("Enter the address: "))
Z = vectorizer.transform([add])
#predict the address
result=model.predict(Z)
#print(result)

address_features = validate_and_extract_address_features(add)
if(address_features['is_valid']==True):
  if(result==[1]):
    print("ETH")
  elif(result==[0]):
    print("BTC")
  elif(result==[2]):
    print("TRX")
else:
  print("Invalid Address")


Enter the address: yw3hd7nd7w3n7xsn w7wnsyduws
Invalid Address


In [2]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [3]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.198s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [4]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & ipv4.icanhazip.com

/bin/bash: line 1: ipv4.icanhazip.com: command not found
npx: installed 22 in 3.476s
your url is: https://petite-hats-listen.loca.lt
